In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d dansbecker/hot-dog-not-hot-dog

In [0]:
!unzip hot-dog-not-hot-dog.zip

In [0]:
!ls seefood/train

In [0]:
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [0]:
TRAIN_DIR = pathlib.Path('seefood/train')
TEST_DIR  = pathlib.Path('seefood/test')

In [0]:
train_list_ds = tf.data.Dataset.list_files(str(TRAIN_DIR/'*/*.jpg'))
test_list_ds = tf.data.Dataset.list_files(str(TEST_DIR/'*/*.jpg'))

In [0]:
for f in test_list_ds.take(5):
  print(f.numpy())

In [0]:
!ls seefood/test/

In [0]:
CLASS_NAMES = ['hot_dog', 'not_hot_dog']
IMAGE_HEIGHT = 160
IMAGE_WIDTH  = 160
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)
BATCH_SIZE = 32

In [0]:
def get_label(file_path):
  print(file_path)
  parts = tf.strings.split(file_path, '/')
  
  if parts[-2] == CLASS_NAMES[0]:
    return 0
  return 1

In [0]:
get_label('seefood/test/hot_dog/422730.jpg')

In [0]:
def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  # img = (img/127.5) - 1
  return tf.image.resize(img, [IMAGE_HEIGHT, IMAGE_WIDTH])

In [0]:
def process_path(file_path):
  label = get_label(file_path=file_path)
  img   = tf.io.read_file(file_path)
  img   = decode_img(img)
  return img, label

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
labeled_ds_train = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
labeled_ds_test = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [0]:
for image, label in labeled_ds_test.take(1):
  plt.imshow(image.numpy())
  plt.title(label.numpy())
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  # ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [0]:
train_ds = prepare_for_training(labeled_ds_train)
test_ds  = prepare_for_training(labeled_ds_test)

In [0]:
image_batch, label_batch = next(iter(train_ds))

In [0]:
plt.imshow(image_batch[1].numpy())
plt.title(label_batch[1].numpy())
print("Image shape: ", image_batch[0].numpy().shape)
print("Label: ", label_batch[0].numpy())

In [0]:
from tensorflow.keras import Sequential, layers, losses, optimizers

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [0]:
base_model.trainable = False

In [0]:
global_average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()

In [0]:
prediction_layer = layers.Dense(1)

In [0]:
model = Sequential([
                    base_model,
                    global_average_pooling_layer,
                    prediction_layer
])

In [0]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'], optimizer=optimizers.Adam(0.001))

In [0]:
model.fit(train_ds,epochs=5, validation_data=test_ds)

In [0]:
!ls

In [0]:
from PIL import Image
import numpy as np

image = Image.open('chili-dog-horiz-a-1600.jpg')

In [0]:
image = image.resize((IMAGE_HEIGHT, IMAGE_WIDTH), resample=Image.NEAREST)
image = np.array(image)
image = image[None,:,:,:]

In [0]:
model.predict(image)